<h3>Importing Modules</h3>

In [1]:
from pyspark.sql import SparkSession, functions as F
import pandas as pd
import warnings
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline

In [2]:
warnings.simplefilter(action='ignore')

# display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<h3>Create SparkSession object</h3>

In [3]:
spark = SparkSession.builder \
    .appName("sensors_realtime_prediction") \
    .master("local[2]") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

<h3>Read data from source</h3>

In [4]:
df = spark.read.format("csv") \
    .option("header", False) \
    .option("inferSchema", True) \
    .option("sep", ",") \
    .load("/home/selcuk/spark/bitirme-projesi/KETI/*/*.csv") \
    .withColumn("file_name", F.input_file_name()) \
    .withColumn("_c0", F.to_timestamp("_c0")) \
    .withColumn("room", F.element_at(F.reverse(F.split(F.col("file_name"), "/")), 2)) \
    .withColumn("sensor", F.regexp_replace(F.element_at(F.reverse(F.split(F.col("file_name"), "/")), 1), ".csv", "")) \
    .withColumnRenamed("_c0", "time") \
    .withColumnRenamed("_c1", "value") \
    .drop("file_name")

df.show(n=10, truncate=False)
df.cache()

+-------------------+------+----+------+
|time               |value |room|sensor|
+-------------------+------+----+------+
|2013-08-23 23:05:03|2287.0|668 |light |
|2013-08-23 23:05:05|1977.0|668 |light |
|2013-08-23 23:05:26|2208.0|668 |light |
|2013-08-23 23:05:30|2267.0|668 |light |
|2013-08-23 23:05:33|2097.0|668 |light |
|2013-08-23 23:05:38|2111.0|668 |light |
|2013-08-23 23:05:43|2257.0|668 |light |
|2013-08-23 23:05:48|2200.0|668 |light |
|2013-08-23 23:05:53|2164.0|668 |light |
|2013-08-23 23:05:58|2183.0|668 |light |
+-------------------+------+----+------+
only showing top 10 rows



DataFrame[time: timestamp, value: double, room: string, sensor: string]

<h3>Creating different dataframes based on sensor type</h3>

In [5]:
##### Divide sensor into dataframes #####
df_pir = df.filter("sensor=='pir'") \
    .withColumn("pir_value", F.col("value")) \
    .withColumn("room_pir", F.col("room")) \
    .withColumn("time_pir", F.col("time")) \
    .drop("sensor", "value", "room", "time")
df_pir.show(5)

+---------+--------+-------------------+
|pir_value|room_pir|           time_pir|
+---------+--------+-------------------+
|      0.0|     648|2013-08-23 19:24:34|
|     29.0|     648|2013-08-23 22:13:24|
|     28.0|     648|2013-08-23 22:13:34|
|     30.0|     648|2013-08-23 22:13:36|
|     30.0|     648|2013-08-23 22:13:37|
+---------+--------+-------------------+
only showing top 5 rows



In [6]:
df_light = df.filter("sensor=='light'") \
    .withColumn("light_value", F.col("value")) \
    .withColumn("room_light", F.col("room")) \
    .withColumn("time_light", F.col("time")) \
    .drop("sensor", "value", "room", "time")
df_light.show(5)

+-----------+----------+-------------------+
|light_value|room_light|         time_light|
+-----------+----------+-------------------+
|     2287.0|       668|2013-08-23 23:05:03|
|     1977.0|       668|2013-08-23 23:05:05|
|     2208.0|       668|2013-08-23 23:05:26|
|     2267.0|       668|2013-08-23 23:05:30|
|     2097.0|       668|2013-08-23 23:05:33|
+-----------+----------+-------------------+
only showing top 5 rows



In [7]:
df_humidity = df.filter("sensor=='humidity'") \
    .withColumn("humidity_value", F.col("value")) \
    .withColumn("room_humidity", F.col("room")) \
    .withColumn("time_humidity", F.col("time")) \
    .drop("sensor", "value", "room", "time")
df_humidity.show(5)

+--------------+-------------+-------------------+
|humidity_value|room_humidity|      time_humidity|
+--------------+-------------+-------------------+
|         50.29|          776|2013-08-23 21:20:29|
|         50.26|          776|2013-08-23 21:20:34|
|         50.26|          776|2013-08-23 21:20:39|
|         50.26|          776|2013-08-23 21:20:44|
|         50.26|          776|2013-08-23 21:20:49|
+--------------+-------------+-------------------+
only showing top 5 rows



In [8]:
df_c02 = df.filter("sensor=='co2'") \
    .withColumn("co2_value", F.col("value")) \
    .withColumn("room_co2", F.col("room")) \
    .withColumn("time_co2", F.col("time")) \
    .drop("sensor", "value", "room", "time")
df_c02.show(5)

+---------+--------+-------------------+
|co2_value|room_co2|           time_co2|
+---------+--------+-------------------+
|    462.0|     415|2013-08-23 16:24:18|
|    468.0|     415|2013-08-23 16:24:23|
|    467.0|     415|2013-08-23 16:24:28|
|    472.0|     415|2013-08-23 16:24:33|
|    482.0|     415|2013-08-23 16:24:38|
+---------+--------+-------------------+
only showing top 5 rows



In [9]:
df_temp = df.filter("sensor=='temperature'") \
    .withColumn("temp_value", F.col("value")) \
    .withColumn("room_temp", F.col("room")) \
    .withColumn("time_temp", F.col("time")) \
    .drop("sensor", "value", "room", "time")
df_temp.show(5)

+----------+---------+-------------------+
|temp_value|room_temp|          time_temp|
+----------+---------+-------------------+
|     24.22|      776|2013-08-23 21:20:29|
|     24.24|      776|2013-08-23 21:20:34|
|     24.24|      776|2013-08-23 21:20:39|
|     24.24|      776|2013-08-23 21:20:44|
|     24.24|      776|2013-08-23 21:20:49|
+----------+---------+-------------------+
only showing top 5 rows



<h3>Joining these dataframes</h3>

In [10]:
df = df_pir.join(df_c02, (df_pir["room_pir"] == df_c02["room_co2"]) & (df_pir["time_pir"] == df_c02["time_co2"]),
                        "inner") \
    .join(df_temp, (df_pir["room_pir"] == df_temp["room_temp"]) & (df_pir["time_pir"] == df_temp["time_temp"]), "inner") \
    .join(df_light, (df_pir["room_pir"] == df_light["room_light"]) & (df_pir["time_pir"] == df_light["time_light"]),
          "inner") \
    .join(df_humidity,
          (df_pir["room_pir"] == df_humidity["room_humidity"]) & (df_pir["time_pir"] == df_humidity["time_humidity"]),
          "inner")

df.count()
# 135386

135386

In [11]:
df.show(3)

+---------+--------+-------------------+---------+--------+-------------------+----------+---------+-------------------+-----------+----------+-------------------+--------------+-------------+-------------------+
|pir_value|room_pir|           time_pir|co2_value|room_co2|           time_co2|temp_value|room_temp|          time_temp|light_value|room_light|         time_light|humidity_value|room_humidity|      time_humidity|
+---------+--------+-------------------+---------+--------+-------------------+----------+---------+-------------------+-----------+----------+-------------------+--------------+-------------+-------------------+
|      0.0|     413|2013-08-25 18:58:04|    459.0|     413|2013-08-25 18:58:04|     23.48|      413|2013-08-25 18:58:04|      103.0|       413|2013-08-25 18:58:04|         52.75|          413|2013-08-25 18:58:04|
|      0.0|     413|2013-08-25 20:22:55|    446.0|     413|2013-08-25 20:22:55|     23.54|      413|2013-08-25 20:22:55|      105.0|       413|2013-

<h3>Dropping Columns</h3>

In [12]:
df = df \
    .withColumn("time", F.col("time_pir")) \
    .withColumn("room", F.col("room_pir")) \
    .drop("value", "room_co2", "time_co2", "room_temp", "time_temp", "room_light", "time_light", "room_humidity",
          "time_humidity", "time_pir", "room_pir")



In [13]:
df.printSchema()

root
 |-- pir_value: double (nullable = true)
 |-- co2_value: double (nullable = true)
 |-- temp_value: double (nullable = true)
 |-- light_value: double (nullable = true)
 |-- humidity_value: double (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- room: string (nullable = true)



In [14]:
df.select("pir_value","co2_value", "temp_value", "light_value", "humidity_value").describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+
|summary|        pir_value|         co2_value|        temp_value|       light_value|    humidity_value|
+-------+-----------------+------------------+------------------+------------------+------------------+
|  count|           135386|            135386|            135386|            135386|            135386|
|   mean|1.666804544044436|399.27423071809494|23.415894774939918|140.57186119687412|56.891765987619664|
| stddev|6.508213709182818|119.08185828746498|11.836374214777713|470.68676287201504| 3.611998136062539|
|    min|              0.0|              62.0|              20.2|               0.0|             42.12|
|    max|             30.0|            1223.0|            579.27|            2397.0|             71.29|
+-------+-----------------+------------------+------------------+------------------+------------------+



<h3>Create Target Feature</h3>

In [15]:
df_last = df.withColumn("pir_value", F.when(F.col("pir_value") > 0, 1) \
                           .otherwise(F.col("pir_value"))) \
    .withColumn("label", F.col("pir_value").cast("int")).drop("pir_value")

df_last.show(3)

+---------+----------+-----------+--------------+-------------------+----+-----+
|co2_value|temp_value|light_value|humidity_value|               time|room|label|
+---------+----------+-----------+--------------+-------------------+----+-----+
|    459.0|     23.48|      103.0|         52.75|2013-08-25 18:58:04| 413|    0|
|    446.0|     23.54|      105.0|         52.43|2013-08-25 20:22:55| 413|    0|
|    455.0|     24.36|      182.0|         50.83|2013-08-25 23:58:53| 413|    0|
+---------+----------+-----------+--------------+-------------------+----+-----+
only showing top 3 rows



In [16]:
df_last.printSchema()

root
 |-- co2_value: double (nullable = true)
 |-- temp_value: double (nullable = true)
 |-- light_value: double (nullable = true)
 |-- humidity_value: double (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- room: string (nullable = true)
 |-- label: integer (nullable = true)



In [18]:
df_last.count()

135386

In [19]:
pandasDF = df_last.toPandas()

In [21]:
pandasDF = pandasDF.sort_values(by=['time'])
pandasDF.head()

,co2_value,temp_value,light_value,humidity_value,time,room,label
103476,387.0,22.62,252.0,52.75,2013-08-23 23:04:53,511,0
14202,579.0,24.37,176.0,49.90,2013-08-23 23:04:57,656A,1
59836,465.0,22.80,165.0,52.40,2013-08-23 23:04:57,644,0
36772,175.0,23.32,191.0,50.32,2013-08-23 23:04:57,648,0
57900,434.0,24.08,11.0,49.94,2013-08-23 23:05:01,564,1


In [ ]:
# pandasDF.to_csv("sensor-data.csv",index=False)

+---------+----------+-----------+--------------+-------------------+----+-----+
|co2_value|temp_value|light_value|humidity_value|               time|room|label|
+---------+----------+-----------+--------------+-------------------+----+-----+
|    419.0|      23.8|        4.0|         50.13|2013-08-27 14:40:43| 456|    0|
|    477.0|     20.78|        3.0|         59.37|2013-08-24 14:26:14| 746|    0|
|    514.0|     22.69|      123.0|         55.89|2013-08-27 00:09:17| 666|    0|
|    401.0|     22.86|        8.0|          57.5|2013-08-28 16:45:53| 640|    0|
|    448.0|     23.23|        5.0|         58.64|2013-08-29 07:01:02| 644|    0|
+---------+----------+-----------+--------------+-------------------+----+-----+
only showing top 5 rows



+---------+----------+-----------+--------------+-------------------+----+-----+-------+---------------+-----------------------------------------------+---------------------------------------+-----------------------------------------+----------+
|co2_value|temp_value|light_value|humidity_value|time               |room|label|roomIdx|ohe_col        |features                                       |rawPrediction                          |probability                              |prediction|
+---------+----------+-----------+--------------+-------------------+----+-----+-------+---------------+-----------------------------------------------+---------------------------------------+-----------------------------------------+----------+
|419.0    |23.8      |4.0        |50.13         |2013-08-27 14:40:43|456 |0    |2.0    |(50,[2],[1.0]) |(54,[0,1,2,3,6],[419.0,23.8,4.0,50.13,1.0])    |[47.574902102442735,2.4250978975572623]|[0.9514980420488547,0.048501957951145244]|0.0       |
|477.0    |20.78

'areaUnderROC'